In [1]:
import numpy as np
import pandas as pd
import matplotlib as matplot
import nltk
import sklearn as sk
import re
import scipy.sparse

# Question 1

In [2]:
en_df_raw = pd.read_csv('data/CONcreTEXT_trial_EN.tsv', sep='\t') # load data files
it_df_raw = pd.read_csv('data/CONcreTEXT_trial_IT.tsv', sep='\t')

en_df = pd.DataFrame()
it_df = pd.DataFrame()

en_df_raw.head()

,TARGET,POS,INDEX,TEXT,MEAN
0,achievement,N,3,"Bring up academic achievements , awards , and ...",3.06
1,achievement,N,9,"Please list people you have helped , your pers...",3.03
2,activate,V,1,Add activated carbon straight to your vodka .,3.83
3,activate,V,15,"Place sensors around your garden , and when a ...",5.51
4,adventure,N,9,Look for a partner that shares your level of a...,2.03


In [3]:
en_df['SENTENCES'] = en_df_raw['TEXT'].apply(lambda sent: sent.strip().lower())
it_df['SENTENCES'] = it_df_raw['TEXT'].apply(lambda sent: sent.strip().lower())

In [4]:
tokenizer = nltk.RegexpTokenizer(r"[A-Za-zÀ-ÖØ-öø-ÿ']+") # because I want to keep apostrophes and accented characters
en_df['WORDS'] = en_df['SENTENCES'].apply(tokenizer.tokenize)
it_df['WORDS'] = it_df['SENTENCES'].apply(tokenizer.tokenize)

In [5]:
en_df['TOKENS'] = en_df['WORDS'].apply(lambda words: ["<s>"] + words + ["</s>"])
it_df['TOKENS'] = it_df['WORDS'].apply(lambda words: ["<s>"] + words + ["</s>"])
en_df['TOKENS'][0]

['<s>',
 'bring',
 'up',
 'academic',
 'achievements',
 'awards',
 'and',
 'other',
 'milestones',
 'in',
 'your',
 'life',
 '</s>']

In [6]:
def ppmi(word1, word2, sentence, W_size=3):
    occurances_word1 = occurance(word1, sentence)
    occurances_word2 = occurance(word2, sentence)
    
    cooccurances = cooccurance(word1, word2, sentence, W_size)
    
    quotient = (cooccurances / (occurances_word1 * occurances_word2) )
    return max(0, np.log2(quotient))

In [7]:
en_vocab = nltk.lm.Vocabulary([word for sentence in en_df['WORDS'] for word in sentence])
len(en_vocab)

644

In [162]:
class PPMI:
    def __init__(self, sentences, window_size=3):
        self.window_size = window_size
        
        self.bagofwords = [word for sentence in sentences for word in sentence]
        self.total_count = len(self.bagofwords)
        self.vocab = nltk.lm.Vocabulary(self.bagofwords)
        self.sentences = sentences
        self.comatrix = pd.DataFrame(0, columns=self.vocab, index=self.vocab, dtype=np.float32)
        self.ppmi_matrix = pd.DataFrame(0, columns=self.vocab, index=self.vocab, dtype=np.float32)
        
    def occurance(self, word):
        num = 0
        den = 0
        for element in self.comatrix[word]:
            num += 1
        
        den = 0
        for col in self.comatrix.index:
            for row in self.comatrix.index:
                den += 1
        
        return num/den
        
    def compute(self):
        tokens = list(self.vocab) # so we can use the symmetry
        
        # map co-occurances
        index = 0
        for word1 in tokens[0:]:
            for word2 in tokens[index+1:]:
                self.comatrix[word1][word2] += self.co(word1, word2)
            index += 1
            
        for col in self.comatrix.index:
            for row in self.comatrix.index:
                self.comatrix[row][col] = self.comatrix[col][row]

        #compute ppmi
        index = 0
        for word1 in tokens[0:]:
            for word2 in tokens[index+1:]:
                if self.comatrix[word1][word2] != 0:
                    numerator = self.comatrix[word1][word2] * self.total_count
                    denominator = self.vocab[word1] * self.vocab[word2]

                    quotient = numerator / denominator
                    self.ppmi_matrix[word1][word2] = max(0,np.lof(quotient))
                
            index += 1
        
        # mirror matrix
        for col in self.ppmi_matrix.index:
            for row in self.ppmi_matrix.index:
                self.ppmi_matrix[row][col] = self.ppmi_matrix[col][row]
                
    def co(self, word1, word2):
        count = 0
        for sentence in self.sentences:
            for index in range(0, len(sentence)):
                if sentence[index] == word1:

                    left_slice_index = max(0, index-self.window_size)
                    right_slice_index = index+self.window_size+1
 
                    for inner_index, inner_word in enumerate(sentence[left_slice_index : right_slice_index]):
                        if inner_word == word2: count += 1

        return count

In [163]:
en_ppmi = PPMI(en_df['TOKENS'][:50])
en_ppmi.compute()

In [164]:
en_ppmi.comatrix

,<s>,bring,up,academic,achievements,awards,and,other,milestones,in,...,has,dozens,places,hide,parents,own,negative,emotions,protect,<UNK>
<s>,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
bring,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
up,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
academic,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
achievements,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
own,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
negative,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
emotions,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
protect,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [165]:
en_ppmi.ppmi_matrix

,<s>,bring,up,academic,achievements,awards,and,other,milestones,in,...,has,dozens,places,hide,parents,own,negative,emotions,protect,<UNK>
<s>,0.000000,1.182985,0.881955,1.182985,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,1.182985,0.000000,0.000000,0.000000,0.000000,0.0
bring,1.182985,0.000000,2.580925,2.881955,2.580925,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
up,0.881955,2.580925,0.000000,2.580925,2.279895,2.580925,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
academic,1.182985,2.881955,2.580925,0.000000,2.580925,2.881955,1.520227,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
achievements,0.000000,2.580925,2.279895,2.580925,0.000000,2.580925,1.219197,2.279895,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
own,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,2.580925,0.000000,0.000000,2.881955,2.881955,0.000000,0.0
negative,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,2.580925,0.000000,2.881955,0.000000,2.881955,2.881955,0.0
emotions,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,2.881955,2.881955,0.000000,2.881955,0.0
protect,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.881955,2.881955,0.000000,0.0


## Question 2

The algorithm I made for forming the PPMI matrix is pretty simple, but I had to optimize some parts to halve the processing time.

First, we tokenize the input sentences. I didn't do this as part of my PPMI class since I don't think it should be encapulated by that class, logically speaking.

Once we have fed the input tokenized sentences into the class constructor, we construct an NLTK Vocabulary object, as well as a Pandas DataFrame filled with 0s.

Then, once compute() is called, we listify the vocabulary so we can use slicing. We slice the two dimensions such that we only compute `[x][y]` instead of both `[x][y]` *and* `[y][x]`. Then, for every combination of two words (excluding symmetrical combinations) we use the sliding window algorithm on each sentence. This ensures that we find all co-occurances for two unique word combinations in each sentence.

The sliding window algorithm is pretty simple:
For each sentence in our corpus, and for each instance of the search word in a given sentence, we go through the neighboring `K` words to see if the second word is found. We avoid overlooking duplicate words in a sentence ince we don't use the index() function.

After we've found all co-occurances and put them in our matrix, we just loop through the same cells as before (the symmetrical subset such that it's from one corner to the diagonal) and compute the probability of the two words independently and use the formula: `log2(cooccurances/probability of individual)`. Then, we just mirror the matrix to the other side of the diagonal.

I think the time complexity is around O(n^4) since even though we split the input token size in half computationally, that's a linear growth which doesn't affect the time complexity increase. It's four nested loops with some if conditionals but I think it works out to n^4. It's not great, but it works!

## Question 3

I would look at the maximum ppmi correspondence for a given word `w` and use the `pd.Series.argmax()` function to find the index of the row that has the highest correlation. Now, our corpus only has 100 sentences and ~670 unique words. I expect this to be a pretty sparse matrix due to this fact. But let's look at two examples! We can see below that 'awards' and the words 'academic', 'milestones' are pretty correlated.

In [ ]:
en_ppmi.matrix['awards'][0:10]

In [ ]:
en_ppmi.co('academic', 'milestones')/( (en_ppmi.vocab['academic'] / len(en_ppmi.vocab) ) * (en_ppmi.vocab['milestones'] / len(en_ppmi.vocab)) )

In [ ]:
en_ppmi.co('academic', 'milestones')

In [153]:
test_sents = [
    ["<s>","nlp", "class", "is", "awesome", "</s>"],
    ["<s>", "nlp", "is", "awesome", "fun", "</s>"],
]

In [156]:
test = PPMI(test_sents, 1)
test.compute()

In [157]:
test.comatrix

,<s>,nlp,class,is,awesome,</s>,fun,<UNK>
<s>,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
nlp,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
class,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
is,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0
awesome,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0
</s>,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
fun,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
<UNK>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
test.ppmi_matrix

,<s>,nlp,class,is,awesome,</s>,fun,<UNK>
<s>,0.000000,1.791759,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
nlp,1.791759,0.000000,1.791759,1.098612,0.000000,0.000000,0.000000,0.0
class,0.000000,1.791759,0.000000,1.791759,0.000000,0.000000,0.000000,0.0
is,0.000000,1.098612,1.791759,0.000000,1.791759,0.000000,0.000000,0.0
awesome,0.000000,0.000000,0.000000,1.791759,0.000000,1.098612,1.791759,0.0
</s>,0.000000,0.000000,0.000000,0.000000,1.098612,0.000000,1.791759,0.0
fun,0.000000,0.000000,0.000000,0.000000,1.791759,1.791759,0.000000,0.0
<UNK>,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [146]:
test.ppmi_matrix

,<s>,nlp,is,a,fun,class,</s>,awesome,i,like,nikko,likes,pizza,<UNK>
<s>,0.000000,1.791759,1.791759,1.791759,0.000000,0.000000,0.000000,1.791759,1.791759,1.791759,1.791759,1.791759,1.791759,0.0
nlp,1.791759,0.000000,2.484907,1.791759,1.791759,0.000000,1.098612,2.484907,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
is,1.791759,2.484907,0.000000,1.791759,1.791759,1.791759,1.098612,2.484907,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
a,1.791759,1.791759,1.791759,0.000000,2.484907,2.484907,1.791759,0.000000,2.484907,2.484907,0.000000,0.000000,0.000000,0.0
fun,0.000000,1.791759,1.791759,2.484907,0.000000,2.484907,1.791759,0.000000,2.484907,2.484907,0.000000,0.000000,0.000000,0.0
class,0.000000,0.000000,1.791759,2.484907,2.484907,0.000000,1.791759,0.000000,0.000000,2.484907,0.000000,0.000000,0.000000,0.0
</s>,0.000000,1.098612,1.098612,1.791759,1.791759,1.791759,0.000000,1.791759,0.000000,0.000000,1.791759,1.791759,1.791759,0.0
awesome,1.791759,2.484907,2.484907,0.000000,0.000000,0.000000,1.791759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
i,1.791759,0.000000,0.000000,2.484907,2.484907,0.000000,0.000000,0.000000,0.000000,3.178054,0.000000,0.000000,0.000000,0.0
like,1.791759,0.000000,0.000000,2.484907,2.484907,2.484907,0.000000,0.000000,3.178054,0.000000,0.000000,0.000000,0.000000,0.0
